In [1]:
import os
import re
import json
from src.constants import VIDEO_GAME_METADATA, VIDEO_GAME_REVIEW, TOY_METADATA, TOY_REVIEW, OUTPUT_META, OUTPUT_REVIEW

In [3]:
%pwd

'f:\\College\\Masters\\Semesters\\Fall 2024\\MLOps\\eCom-chat'

In [2]:
os.chdir('../')

# Review Data

### View Data

In [5]:
with open(VIDEO_GAME_REVIEW, 'r') as fp:
    for line in fp:
        print(json.dumps(json.loads(line.strip()), indent=4))
        break

{
    "rating": 4.0,
    "title": "It\u2019s pretty sexual. Not my fav",
    "text": "I\u2019m playing on ps5 and it\u2019s interesting.  It\u2019s unique, massive, and has a neat story.  People are freaking out angry about this game.  I don\u2019t think it\u2019s a top 10 game but it\u2019s definitely a good game on ps5 (played at launch).",
    "images": [],
    "asin": "B07DJWBYKP",
    "parent_asin": "B07DK1H3H5",
    "user_id": "AGCI7FAH4GL5FI65HYLKWTMFZ2CQ",
    "timestamp": 1608186804795,
    "helpful_vote": 0,
    "verified_purchase": true
}


In [6]:
with open(TOY_REVIEW, 'r') as fp:
    for line in fp:
        print(json.dumps(json.loads(line.strip()), indent=4))
        break

{
    "rating": 5.0,
    "title": "Granddaughters love them!",
    "text": "I purchased several of these for my granddaughters for Xmas & they absolutely love them. In fact so do their brothers. Lol. Be aware that there are tiny magnets in them so that you can perch the ppl onto the animals & keep them there. I almost didn\u2019t buy them bc of that given all the news stories about kids and magnets, but the magnets are INSIDE the toys. So as long as ur kids aren\u2019t eating their toys you should be okay, but be careful around your pets so that the pets don\u2019t try to eat them.  My grandkids adore all things Schleich as do I bc I bought it for their parents too decades ago.  Schleich is well made & lasts forever.",
    "images": [],
    "asin": "B09QH7QJS7",
    "parent_asin": "B09QH7QJS7",
    "user_id": "AFKZENTNBQ7A7V7UXW5JJI6UGRYQ",
    "timestamp": 1677939664713,
    "helpful_vote": 0,
    "verified_purchase": true
}


### Merge Data

In [30]:
# Open output file in write mode
with open(OUTPUT_REVIEW, 'w') as out_fp:
    
    # Open the video games review file and write its content to the output file
    with open(VIDEO_GAME_REVIEW, 'r') as video_fp:
        for line in video_fp:
            out_fp.write(line)
    
    # Open the toys review file and write its content to the output file
    with open(TOY_REVIEW, 'r') as toy_fp:
        for line in toy_fp:
            out_fp.write(line)

# Meta Data

### View Data

In [7]:
with open(VIDEO_GAME_METADATA, 'r') as fp:
    for line in fp:
        print(json.dumps(json.loads(line.strip()), indent=4))
        break

{
    "main_category": "Video Games",
    "title": "Dash 8-300 Professional Add-On",
    "average_rating": 5.0,
    "rating_number": 1,
    "features": [
        "Features Dash 8-300 and 8-Q300 ('Q' rollout livery)",
        "Airlines - US Airways, South African Express, Bahamasair, Augsburg Airways, Lufthansa Cityline, British Airways (Union Jack), British European, FlyBe, Intersky, Wideroe, Iberia, Tyrolean, QantasLink, BWIA",
        "Airports include - London City, Frankfurt, Milan and Amsterdam Schipol",
        "Includes PSS PanelConfig and LoadEdit tools"
    ],
    "description": [
        "The Dash 8-300 Professional Add-On lets you pilot a real commuter special. Fly two versions of the popular Dash 8-300 in a total of 17 different liveries. The Dash 8-300 is one of the most popular short-haul aircraft available and this superbly modelled version from acclaimed aircraft developers PSS is modelled in two versions with a total of 17 different liveries. The package also includes 

In [8]:
with open(TOY_METADATA, 'r') as fp:
    for line in fp:
        print(json.dumps(json.loads(line.strip()), indent=4))
        break

{
    "main_category": "Toys & Games",
    "title": "KUNGOON Happy Anniversary Balloon Banner,Wedding Anniversary Party Decorations,Love Party and Anniversary Party Supplies,16 Inch Gold Aluminum Foil.",
    "average_rating": 4.5,
    "rating_number": 241,
    "features": [],
    "description": [],
    "price": null,
    "images": [
        {
            "thumb": "https://m.media-amazon.com/images/I/41enwmDCoZL._AC_US40_.jpg",
            "large": "https://m.media-amazon.com/images/I/41enwmDCoZL._AC_.jpg",
            "variant": "MAIN",
            "hi_res": "https://m.media-amazon.com/images/I/51tskkWgFmL._AC_SL1000_.jpg"
        },
        {
            "thumb": "https://m.media-amazon.com/images/I/414NxCPIOmL._AC_US40_.jpg",
            "large": "https://m.media-amazon.com/images/I/414NxCPIOmL._AC_.jpg",
            "variant": "PT01",
            "hi_res": "https://m.media-amazon.com/images/I/51pCkL9l5LL._AC_SL1000_.jpg"
        },
        {
            "thumb": "https://m.media-ama

### Merge and Clean Data

In [23]:
def clean_field_name_for_bigquery(field_name):
    """Clean the field name to meet BigQuery requirements."""
    # Replace spaces and special characters with underscores
    cleaned_name = re.sub(r'[^a-zA-Z0-9_]', '_', field_name)
    # Convert to lower case
    cleaned_name = cleaned_name.lower()
    # Truncate the field name to 300 characters if necessary
    return cleaned_name[:300]

In [25]:
def process_field_names(data, existing_fields=None):
    """Recursively rename fields to meet BigQuery's requirements and remove empty string field names."""
    if existing_fields is None:
        existing_fields = set()
        
    if isinstance(data, dict):
        cleaned_data = {}
        for key, value in data.items():
            # Clean the field name to comply with BigQuery
            cleaned_key = clean_field_name_for_bigquery(key)
            
            # Ensure uniqueness of the cleaned field name
            if cleaned_key in existing_fields:
                suffix = 1
                new_key = f"{cleaned_key}_{suffix}"
                while new_key in existing_fields:
                    suffix += 1
                    new_key = f"{cleaned_key}_{suffix}"
                cleaned_key = new_key
            
            # Add cleaned key to existing fields
            existing_fields.add(cleaned_key)
            
            # Recursively process sub-fields
            cleaned_data[cleaned_key] = process_field_names(value, existing_fields)
        
        # Remove any keys that are empty strings
        return {k: v for k, v in cleaned_data.items() if k != ""}
    
    elif isinstance(data, list):
        # Recursively process each item in the list
        return [process_field_names(item, existing_fields) for item in data]
    
    else:
        # Return the data as is if it's not a dict or list
        return data

In [26]:
def process_file_and_save(file_path, output_fp):
    """Process a JSONL file, clean field names, remove empty field names, and save to the output file."""
    with open(file_path, 'r') as fp:
        for line in fp:
            data = json.loads(line.strip())
            cleaned_data = process_field_names(data)
            # Write the cleaned and renamed data back to the output file
            output_fp.write(json.dumps(cleaned_data) + '\n')

In [28]:
# Open the output file to save the cleaned data
with open(OUTPUT_META, 'w') as out_fp:
    # Process both files and save cleaned data for BigQuery
    process_file_and_save(VIDEO_GAME_METADATA, out_fp)
    process_file_and_save(TOY_METADATA, out_fp)